In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

from gammapy.data import DataStore
from gammapy.scripts import SpectrumAnalysisIACT

# Convenience classes to define analsys inputs
# At some point we'll add a convenience layer to run the analysis starting from a plain text config file.
from gammapy.utils.energy import EnergyBounds
from gammapy.image import SkyImage
from gammapy.spectrum import models
from regions import CircleSkyRegion
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

from make_veritas_obs_hdu_index import create_obs_hdu_index

In [ ]:
store_dir = './'
dl3Dir = "./VEGAS/DL3/"


dl3Files = [join(dl3Dir, f) for f in listdir(dl3Dir) if isfile(join(dl3Dir, f)) and f.endswith("fits")]
create_obs_hdu_index(dl3Files)

data_store = DataStore.from_dir(store_dir)
obs_id = data_store.obs_table['OBS_ID'].data
print("Use observations {}".format(obs_id))

obs_list = data_store.obs_list(obs_id)

In [ ]:
crab_pos = SkyCoord.from_name('crab')
on_region = CircleSkyRegion(crab_pos, 0.1 * u.deg)

model = models.LogParabola(
    alpha = 2.3,
    beta = 0,
    amplitude = 1e-11 * u.Unit('cm-2 s-1 TeV-1'),
    reference = 1 * u.TeV,
)

# model = models.PowerLaw(
#     index = 2.3,
#     amplitude = 1e-11 * u.Unit('cm-2 s-1 TeV-1'),
#     reference = 1 * u.TeV,
# )

flux_point_binning = EnergyBounds.equal_log_spacing(0.1585, 3.9811, 15, u.TeV)

exclusion_mask = SkyImage.read('$GAMMAPY_EXTRA/datasets/exclusion_masks/tevcat_exclusion.fits')


In [ ]:
config = dict(
    outdir = None,
    background = dict(
        on_region=on_region,
        exclusion_mask=exclusion_mask,
        min_distance = 0.1 * u.rad,
    ),
    extraction = dict(containment_correction=False),
    fit = dict(
        model=model,
        stat='wstat',
        forward_folded=True,
        fit_range = flux_point_binning[[0, -1]]
    ),
    fp_binning=flux_point_binning
)

In [ ]:
ana = SpectrumAnalysisIACT(
    observations=obs_list,
    config=config,
)
ana.run()

In [ ]:
print(ana.fit.result[0])

In [ ]:
ax = ana.spectrum_result.plot(
    energy_range=ana.fit.fit_range,
    energy_power=2,
    flux_unit='erg-1 cm-2 s-1',
    fig_kwargs=dict(figsize = (8,8)),
)
ax[1].set_ylim(-2, 2)

In [ ]:
ff = fits.open("VEGAS/CrabDL3medPoint_s6.fits")
points = ff[1].data
fit = ff[2].data

points = points[points['Significance'] > 2]
points = points[points['On'] > 2]

energy = np.array(points['Energy']) * u.TeV
flux = np.array(points['Flux'])*u.Unit("TeV-1 m-2 s-1")
fluxE = np.array(points['FluxError'])*u.Unit("TeV-1 m-2 s-1")

In [ ]:
ax = ana.spectrum_result.plot(
    energy_range=ana.fit.fit_range,
    energy_power=2,
    flux_unit='erg-1 cm-2 s-1',
    fig_kwargs=dict(figsize=(8, 8)), )
ax[1].set_ylim(-2, 2)

ax[0].errorbar(
    energy.value, (flux * energy**2).to(u.Unit("erg cm-2 s-1")).value,
    yerr=(fluxE * energy**2).to(u.Unit("erg cm-2 s-1")).value,
    ls="",
    marker="_")

ax[0].plot(energy.value,
           (fit["Normalization"][0] * u.Unit("TeV-1 m-2 s-1") * energy.value**
            fit["Index"][0] * energy**2).to(u.Unit("erg cm-2 s-1")).value)

ax[0].set_ylim(ymin=5e-12)

# plt.savefig("Plots/spec_comp.pdf")

In [ ]:
import plottingFunctions as pF

In [ ]:
spec = pF.spectrumPlotter()
spec.readVEGASFits("VEGAS/CrabDL3medPoint_s6.fits")
spec.e2fluxUnits = u.Unit("erg cm-2 s-1")
spec.plotSpectrum()


In [ ]:
print points['EBinLowEdge']
print points['EBinHiEdge']